In [593]:
def extrair_dados(linha):
    linha = linha.replace(', ', ' - ')
    linha = linha.replace('"', '')
    return linha.split(',')[:-1]

In [594]:
dBairro = sc.textFile('data/bairros.csv')
dBairro = dBairro.map(extrair_dados)
hBairro = dBairro.first()
bairro =  dBairro.filter(lambda dBairro: dBairro != hBairro)

In [595]:
dPopulacao = sc.textFile('data/populacao.json')
dPopulacao = dPopulacao.map(extrair_dados)
hPopulacao = dPopulacao.first()
populacao = dPopulacao.filter(lambda dPopulacao: dPopulacao != hPopulacao)

In [596]:
dEventoFluxo = sc.textFile('data/eventos_de_fluxo.csv')
dEventoFluxo = dEventoFluxo.map(extrair_dados)
hEventoFluxo = dEventoFluxo.first()
eventoFluxo = dEventoFluxo.filter(lambda dEventoFluxo: dEventoFluxo != hEventoFluxo)

In [597]:
dConcorrente = sc.textFile('data/concorrentes.csv')
dConcorrente = dConcorrente.map(extrair_dados)
hConcorrente = dConcorrente.first()
concorrente = dConcorrente.filter(lambda dConcorrente: dConcorrente != hConcorrente)

### Qual é a faixa de preço praticada pelos concorrentes ? 
### Como é o fluxo de pessoas nesses locais ? 
### Qual é a população e a densidade demográfica dos bairros onde os concorrentes estão ?

Arquivo populacao.csv para relacionar com o código do bairro obter quantidade de habitantes por bairro.


In [598]:
concorrente.take(5)

[['431962533652067',
  'Boizão Lanches',
  'Bar',
  '2',
  '13190-000 Monte Mor',
  'Monte Mor',
  'SP'],
 ['1663855903830869',
  'Bar do Serjão',
  'Bar',
  '0',
  'Rua das Dracenas - Americana',
  'Americana',
  'SP'],
 ['567824576564110',
  'Recanto Do Kuca',
  'Restaurant',
  '0',
  'Jarinu',
  'Jarinu',
  'SP'],
 ['202740866540615',
  'Dedé Abelhuda',
  'Grocery Store',
  '0',
  'SP - 13150000 Cosmópolis',
  'Cosmópolis',
  'SP'],
 ['1784900838394305',
  'Tenshi Sushi Boteco Itu',
  'Sushi Restaurant',
  '3',
  'Av. Plaza - 170 - 13302-100 Itu',
  'Itu',
  'SP']]

In [599]:
def converter_tipo(valor, padrao, *tipos):
    for t in tipos:
        try:
            return t(valor)
        except ValueError:
            continue
    return padrao

In [600]:
index_categoria = hConcorrente.index('categoria')
index_codigo = hConcorrente.index('codigo')
index_faixa_preco = hConcorrente.index('faixa_preco')

In [601]:
categorias = concorrente.map(lambda x: (x[index_categoria].split(' - '), 
                                        converter_tipo(x[index_faixa_preco], 0, int, float)))

In [602]:
categorias.take(15)
#categorias.count()

[(['Bar'], 2),
 (['Bar'], 0),
 (['Restaurant'], 0),
 (['Grocery Store'], 0),
 (['Sushi Restaurant'], 3),
 (['Restaurant', 'Other'], 1),
 (['Grocery Store'], 0),
 (['Pub'], 0),
 (['Grocery Store'], 0),
 (['Restaurant'], 0),
 (['Restaurant'], 1),
 (['Bar'], 1),
 (['Beer Bar', 'Event'], 2),
 (['Restaurant', 'Hotel Resort'], 2),
 (['Pub', 'Just For Fun'], 2)]

In [603]:
test = {'a': [23]}
test.get('a').append(1)
test


{'a': [23, 1]}

In [667]:
faixa_preco_concorrente = {}
lista_faixa = []
for local in categorias.collect():
    for categ in local[0]:
        concorrente = faixa_preco_concorrente.get(categ) or []
        if (concorrente != categ):
            concorrente.append(local[1])
            faixa_preco_concorrente.update({categ: concorrente})
            minimo = min(concorrente)
            maximo = max(concorrente)
            faixa_preco_concorrente.update({'faixa_preco': {"min": minimo, "max": maximo}})
        else:
            concorrente = []
    
faixa_preco_concorrente

{'Accessories': [2],
 'Adult Entertainment Service': [2, 4, 4, 1, 3, 3, 4],
 'Agricultural Service': [1, 0],
 'Agriculture Company': [0],
 'Airport': [2],
 'Alternative & Holistic Health Service': [0],
 'American Restaurant': [2, 1, 3, 0, 2, 2, 1],
 'Amusement & Theme Park': [2, 1, 1],
 'Antique Store': [2],
 'Apartment & Condo Building': [1],
 'Apparel & Clothing': [0],
 'Appliances': [0],
 'Arabian Restaurant': [3, 0],
 'Arena & Sports Venue': [1, 1, 1],
 'Argentinian Restaurant': [3, 4, 0],
 'Arts & Crafts Store': [0],
 'Arts & Entertainment': [4,
  4,
  1,
  2,
  2,
  1,
  1,
  1,
  2,
  2,
  2,
  1,
  1,
  1,
  2,
  1,
  1,
  2,
  2,
  2,
  1,
  2,
  2,
  2,
  1,
  4,
  3,
  1,
  2,
  1,
  1,
  2,
  2,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  2,
  2,
  1,
  2,
  2,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  1,
  2,
  2,
  4,
  1,
  0,
  2,
  2,
  0,
  2,
  2,
  3,
  2,
  1,
  1,
  1

In [650]:
fpc = sc.parallelize(faixa_preco_concorrente)

In [651]:
fpc

ParallelCollectionRDD[419] at parallelize at PythonRDD.scala:489

In [652]:
fpc = fpc.map(lambda x: (x[index_categoria], min(x[1]), max(x[1]))).groupByKey()

In [653]:
fpc.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 370.0 failed 1 times, most recent failure: Lost task 1.0 in stage 370.0 (TID 1138, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark-2.2.1/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/opt/spark-2.2.1/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark-2.2.1/python/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark-2.2.1/python/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark-2.2.1/python/pyspark/rdd.py", line 346, in func
    return f(iterator)
  File "/opt/spark-

In [654]:
faixa_preco_concorrente = faixa_preco_concorrente   #.groupByKey()  #.mapValues(min)
# faixa_preco = faixa_preco.sortBy(lambda x: x[1], ascending=False)

In [655]:
# faixa_preco_concorrente = faixa_preco.map(lambda x: (x[index_categoria], x[index_faixa_preco]))

In [656]:
faixa_preco_concorrente = faixa_preco.groupByKey()  #.mapValues(max)

AttributeError: 'NoneType' object has no attribute 'groupByKey'